In [2]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [3]:
embeddings = np.load("embeddings_sample.npy")
vocab = pickle.load(open("data/vocab_full.pkl",'rb'))

t_pos = pd.read_table("data/train_pos.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_pos['label'] = 1
t_neg = pd.read_table("data/train_neg.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_neg['label'] = -1
df = pd.concat((t_pos,t_neg))
df.shape, embeddings.shape, len(vocab)

((196970, 2), (101298, 20), 101298)

In [4]:
max_len = max([len(tweet.split(' ')) for tweet in df.tweet])
print(f'{max_len=}')

max_len=1626


In [5]:
tokenized_tweets_np = np.zeros((len(df),max_len))
for i, tweet in enumerate(tqdm(df.tweet)):
    tweet_len = len(tweet.split(' '))
    tweet_embedding=np.zeros((tweet_len,20))
    for j, word in enumerate(tweet.split(' ')):
        if word not in vocab.keys():
            continue
        tweet_embedding[j,:] = embeddings[vocab.get(word),:]
    
    tokenized_tweets_np[i,:tweet_len] = tweet_embedding.sum(1)
tokenized_tweets_np.shape

100%|██████████| 196970/196970 [00:09<00:00, 21621.18it/s]


(196970, 1626)

In [6]:
X = tokenized_tweets_np
y = df.label.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((157576, 1626), (39394, 1626), (157576,), (39394,))

In [10]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)

/home/tcastigl/miniconda3/envs/ML_env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [11]:
print(f'logreg training accuracy = {logreg.score(X_train, y_train):.4f}')
print(f'logreg val accuracy = {logreg.score(X_val, y_val):.4f}')

logreg training accuracy = 0.5493
logreg val accuracy = 0.5469


In [12]:
SVC = LinearSVC(verbose=1, max_iter=2000)
SVC.fit(X_train,  y_train)

[LibLinear]........................................................................................................................................................................................................
optimization finished, #iter = 2000

Using -s 2 may be faster (also see FAQ)

Objective value = -155748.212951
nSV = 157509


/home/tcastigl/miniconda3/envs/ML_env/lib/python3.11/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=2000, verbose=1)

In [13]:
print(f'SVM training accuracy = {SVC.score(X_train, y_train):.4f}')
print(f'SVM val accuracy = {SVC.score(X_val, y_val):.4f}')

SVM training accuracy = 0.5492
SVM val accuracy = 0.5471
